# 🧪 Running GitLab CI Locally with `gitlab-runner`

This example shows how to test a `.gitlab-ci.yml` pipeline job locally using `gitlab-runner` in shell mode. We'll generate a test script and CI config, then run it without needing a GitLab server.

In [9]:
import os
import subprocess
from pathlib import Path

env = os.environ.copy()
env["PATH"] = f"{home}/bin:" + env["PATH"]
env["SHELL"] = "/bin/bash"  # ← required to make shell executor work!

# Define project path
home = Path.home()
proj = home / "gitlab_demo"
proj.mkdir(parents=True, exist_ok=True)

# Add $HOME/bin to PATH (for gitlab-runner)
env = os.environ.copy()
env["PATH"] = f"{home}/bin:" + env["PATH"]

# Optional cleanup
if (proj / ".git").exists():
    subprocess.run("rm -rf .git", shell=True, cwd=proj)

# Write test file
(proj / "test_sample.py").write_text("""\
def add(a, b):
    return a + b

def test_add():
    assert add(2, 2) == 4
""")

# Write GitLab CI YAML
(proj / ".gitlab-ci.yml").write_text("""\
stages:
  - test

pytest:
  stage: test
  script:
    - pip install pytest
    - pytest
""")

# Initialize git repo and commit
subprocess.run("git config --global init.defaultBranch main", shell=True, env=env)
subprocess.run("git init", shell=True, cwd=proj, env=env)
subprocess.run("git config user.name 'CI Tester'", shell=True, cwd=proj, env=env)
subprocess.run("git config user.email 'ci@test.local'", shell=True, cwd=proj, env=env)
subprocess.run("git add .", shell=True, cwd=proj, env=env)
subprocess.run("git commit -m 'initial commit'", shell=True, cwd=proj, env=env)

# Run GitLab job
print(f"\n🚀 Running GitLab CI job in {proj}...\n")
result = subprocess.run("gitlab-runner exec docker --docker-image python:3.10 pytest", shell=True, cwd=proj, env=env)

# Outcome
if result.returncode == 0:
    print("\n✅ GitLab CI job executed successfully.")
else:
    print("\n❌ GitLab CI job failed.")


Initialized empty Git repository in /home/rolan/gitlab_demo/.git/
[main (root-commit) ae21162] initial commit
 5 files changed, 20 insertions(+)
 create mode 100644 .gitlab-ci.yml
 create mode 160000 builds/0/project-0
 create mode 100644 builds/0/project-0.tmp/git-template/config
 create mode 100644 builds/0/project-0.tmp/gitlab_runner_env
 create mode 100644 test_sample.py

🚀 Running GitLab CI job in /home/rolan/gitlab_demo...

Running with gitlab-runner 16.9.1 (782c6ecb)
Preparing the "docker" executor
Using Docker executor with image python:3.10 ...
Using helper image:  registry.gitlab.com/gitlab-org/gitlab-runner/gitlab-runner-helper:x86_64-v16.9.1


hint: You've added another git repository inside your current repository.
hint: Clones of the outer repository will not contain the contents of
hint: the embedded repository and will not know how to obtain it.
hint: If you meant to add a submodule, use:
hint: 
hint: 	git submodule add <url> builds/0/project-0
hint: 
hint: If you added this path by mistake, you can remove it from the
hint: index with:
hint: 
hint: 	git rm --cached builds/0/project-0
hint: 
hint: See "git help submodule" for more information.
Runtime platform                                    arch=amd64 os=linux pid=21732 revision=782c6ecb version=16.9.1
fatal: ambiguous argument 'HEAD~1': unknown revision or path not in the working tree.
Use '--' to separate paths from revisions, like this:
'git <command> [<revision>...] -- [<file>...]'


Pulling docker image registry.gitlab.com/gitlab-org/gitlab-runner/gitlab-runner-helper:x86_64-v16.9.1 ...
Using docker image sha256:24432bb8b93507e7bc4b87327c24317029f1ea0315abf1bc7f71148f2555d681 for registry.gitlab.com/gitlab-org/gitlab-runner/gitlab-runner-helper:x86_64-v16.9.1 with digest registry.gitlab.com/gitlab-org/gitlab-runner/gitlab-runner-helper@sha256:24432bb8b93507e7bc4b87327c24317029f1ea0315abf1bc7f71148f2555d681 ...
Pulling docker image python:3.10 ...
Using docker image sha256:d188cfc2b726fa69a76d1eaebb50a80bc61fa312a4aad6bbda145b0ec77dcf75 for python:3.10 with digest python@sha256:d188cfc2b726fa69a76d1eaebb50a80bc61fa312a4aad6bbda145b0ec77dcf75 ...
Preparing environment
Running on runner--project-0-concurrent-0 via White-WIN...
Getting source from Git repository
Fetching changes...
Initialized empty Git repository in /builds/project-0/.git/
Created fresh repository.
Checking out ae211622 as detached HEAD (ref is main)...

Skipping Git submodules setup
Executing "step_